In [1]:
from datetime import datetime, timedelta
import pandas as pd
import logging
from binance.um_futures import UMFutures
from binance.lib.utils import config_logging
import time
from binance.spot import Spot


def pull_binance_data(interval = '5m'):
    exchange_key = 'HHwPdDKvCO3zxbLqQbjxcb0N2wDNDs2aD5aIz3M3GsuiIntsgfV0wWWIaqfnKriw'
    end_time = round(time.time() * 1000)
    end_datetime = datetime.fromtimestamp(end_time / 1000.0)
    config_logging(logging, logging.DEBUG)

    um_futures_client = UMFutures(key = exchange_key)

    kline = um_futures_client.klines("BTCUSDT", interval, end_time = end_time, limit = 1000)

    columns = ['open_time', 'open', 'high', 'low', 'close', 'volume',
               'close_time', 'quote_asset_volume', 'number_of_trades',
               'taker_buy_base_volume', 'taker_buy_quote_asset_volume',
               'ignore']

    df = pd.DataFrame(kline, columns = columns)
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    df = df.drop(['taker_buy_base_volume'], axis = 1)
    return df

def pull_binance_data_spot(interval = '5m'):
    exchange_key = 'HHwPdDKvCO3zxbLqQbjxcb0N2wDNDs2aD5aIz3M3GsuiIntsgfV0wWWIaqfnKriw'
    end_time = round(time.time() * 1000)
    end_datetime = datetime.fromtimestamp(end_time / 1000.0)
    # config_logging(logging, logging.DEBUG)

    client = Spot(api_key = exchange_key)

    kline = client.klines("BTCUSDT", interval, endTime = end_time, limit = 1000)

    columns_spot = ['open_time', 'open', 'high', 'low', 'close', 'volume',
                    'close_time', 'quote_asset_volume', 'number_of_trades',
                    'taker_buy_base_volume', 'taker_buy_quote_asset_volume', 'ignore']

    df = pd.DataFrame(kline, columns = columns_spot)
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    df = df.drop(['taker_buy_base_volume'], axis = 1)

    return df

In [2]:
df_5m = pull_binance_data('5m')

2023-12-04 23:06:13.880 UTC DEBUG binance.api: url: https://fapi.binance.com/fapi/v1/klines
2023-12-04 23:06:14.953 UTC DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): fapi.binance.com:443
2023-12-04 23:06:15.923 UTC DEBUG urllib3.connectionpool: https://fapi.binance.com:443 "GET /fapi/v1/klines?symbol=BTCUSDT&interval=5m&end_time=1701731173880&limit=1000 HTTP/1.1" 200 None
2023-12-04 23:06:16.237 UTC DEBUG binance.api: raw response from server:[[1701431400000,"38589.90","38595","38563","38563","494.898",1701431699999,"19093496.14070",6553,"112.623","4345462.17560","0"],[1701431700000,"38563","38598.10","38555.50","38578","422.263",1701431999999,"16287847.78600",6049,"234.505","9045624.70840","0"],[1701432000000,"38578","38629.90","38578","38614.80","632.016",1701432299999,"24400586.32090",6869,"462.909","17870986.16190","0"],[1701432300000,"38614.80","38615.60","38548","38548","603.347",1701432599999,"23278633.19170",6899,"125.442","4840434.44620","0"],[1701432600000,

In [4]:

df_prev = pd.read_feather('../data/df_btc_with_features_5m_spot.feather')
columns = ['open_time', 'open', 'high', 'low', 'close', 'volume',
           'close_time', 'quote_asset_volume', 'number_of_trades',
           'taker_buy_quote_asset_volume', 'ignore']

try:
    df_prev = df_prev.drop(['taker_buy_base_volume'], axis = 1)
except:
    print(f'taker buy base volume not in df_prev')

df_live = pull_binance_data_spot('5m')

df_prev = df_prev[columns]
df_live['open_time'] = pd.to_datetime(df_live['open_time'], unit='ms')
df_live['close_time'] = pd.to_datetime(df_live['close_time'], unit='ms')
time_cols = ['open_time', 'close_time']
float_cols = [x for x in columns if x not in time_cols]
df_live[float_cols] = df_live[float_cols].astype(float)
df_live['token'] = 'BTCUSDT'
df_prev = df_prev.iloc[-30000:, :]
df = pd.concat([df_prev, df_live], axis=0, ignore_index=True)

2023-12-04 23:06:34.224 UTC DEBUG binance.api: url: https://api.binance.com/api/v3/klines
2023-12-04 23:06:34.313 UTC DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): api.binance.com:443
2023-12-04 23:06:34.610 UTC DEBUG urllib3.connectionpool: https://api.binance.com:443 "GET /api/v3/klines?symbol=BTCUSDT&interval=5m&endTime=1701731194224&limit=1000 HTTP/1.1" 200 52867
2023-12-04 23:06:34.627 UTC DEBUG binance.api: raw response from server:[[1701431400000,"38593.07000000","38598.98000000","38569.47000000","38569.48000000","65.16175000",1701431699999,"2514540.62971910",3530,"13.48160000","520219.42638770","0"],[1701431700000,"38569.47000000","38595.51000000","38563.93000000","38580.17000000","71.68242000",1701431999999,"2765246.62017170",3143,"34.40123000","1327016.75051050","0"],[1701432000000,"38580.17000000","38629.65000000","38580.16000000","38618.12000000","126.37701000",1701432299999,"4880038.73899960",3886,"59.34523000","2291412.82802100","0"],[1701432300000,"386

In [6]:
df.tail()

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_quote_asset_volume,ignore,token
30995,2023-12-04 22:45:00,42077.76,42077.76,41855.27,41902.64,417.90192,2023-12-04 22:49:59.999,1.753286e+07,12344.0,6.923729e+06,0.0,BTCUSDT
30996,2023-12-04 22:50:00,41902.65,41987.00,41861.86,41924.83,258.76866,2023-12-04 22:54:59.999,1.084391e+07,6537.0,6.091119e+06,0.0,BTCUSDT
30997,2023-12-04 22:55:00,41924.82,41974.79,41914.76,41950.60,127.96283,2023-12-04 22:59:59.999,5.367402e+06,3700.0,2.827605e+06,0.0,BTCUSDT
30998,2023-12-04 23:00:00,41950.60,41950.60,41814.91,41910.27,406.71723,2023-12-04 23:04:59.999,1.702889e+07,8438.0,6.708312e+06,0.0,BTCUSDT
30999,2023-12-04 23:05:00,41910.27,41910.40,41891.11,41899.13,47.03084,2023-12-04 23:09:59.999,1.970750e+06,1125.0,5.068014e+05,0.0,BTCUSDT


In [12]:
df_1m.tail(1100)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_quote_asset_volume,ignore
0,2023-12-02 17:08:00,38746.20,38747.70,38735.90,38735.90,57.665,2023-12-02 17:08:59.999,2234141.05580,783,993679.25370,0
1,2023-12-02 17:09:00,38735.90,38736,38730,38735.90,71.879,2023-12-02 17:09:59.999,2784131.67080,687,415764.41830,0
2,2023-12-02 17:10:00,38736,38739.30,38735.90,38739.30,27.203,2023-12-02 17:10:59.999,1053807.71910,501,691676.49210,0
3,2023-12-02 17:11:00,38739.30,38745.50,38739.20,38745.50,31.075,2023-12-02 17:11:59.999,1203892.81760,620,965240.51280,0
4,2023-12-02 17:12:00,38745.50,38745.60,38745.40,38745.60,17.960,2023-12-02 17:12:59.999,695869.19070,367,565645.89370,0
...,...,...,...,...,...,...,...,...,...,...,...
995,2023-12-03 09:43:00,39440.50,39440.50,39422.60,39433,103.903,2023-12-03 09:43:59.999,4097061.74410,1183,1968610.25150,0
996,2023-12-03 09:44:00,39432.90,39433.80,39428.50,39428.50,19.656,2023-12-03 09:44:59.999,775085.58440,380,134070.92530,0
997,2023-12-03 09:45:00,39428.50,39428.60,39428.30,39428.30,24.033,2023-12-03 09:45:59.999,947584.23000,369,173051.97670,0
998,2023-12-03 09:46:00,39428.40,39428.60,39415.80,39415.80,53.191,2023-12-03 09:46:59.999,2097093.66460,844,684830.93720,0
